# My Great Capstone Project

In this notebook we are collecting the necessary data for our data science capstone project. First we scrape the wikipedia page by using requests to retrieve the hmtl which then is transformed into a BeautifulSoup object. From there we extract the extract the needed information and store the data in a pandas dataframe.

## Collecting the data

First thing to do is to load the necessary packages. Make sure you have install all of them. It happens that you have to install the "lxml" package separately.

In [1]:
# %pip install lxml
# %pip install bs4
# %pip install requests
# %pip install pandas
from bs4 import BeautifulSoup
import pandas as pd
import requests

Now we are ready to get the data from the link and convert it into a BeautifulSoup object.

In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_html = requests.get(wiki_url)
soup = BeautifulSoup(raw_html.text)

Firstly we create a list. After finding the table and table data we create a dictionary called cell having 3 keys PostalCode, Borough and Neighborhood. In the end we create a dataframe with the list.

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

toronto_df=pd.DataFrame(table_contents)

Let's check what we've got:

In [4]:
toronto_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


This dataframe looks good. But taking a closer look we see that we can clean the 'Borugh' column better. For example we replace 'Downtown TorontoStn A PO Boxes25 The Esplanade' with 'Downtown Toronto Stn A'

In [5]:

toronto_df['Borough']=toronto_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

A last check that all postal codes are unique and that no cell has a borough but a Not assigned  neighborhood.

In [6]:
print('All postal codes are unique:', len(toronto_df.PostalCode.unique())==toronto_df.shape[0])

not_assigned_n = False if sum(toronto_df.Neighborhood=='Not assigned')==0 else True
print("A cell has a borough but a 'Not assigned' neighborhood:", not_assigned_n)


All postal codes are unique: True
A cell has a borough but a 'Not assigned' neighborhood: False


Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. We will use the Geocoder.

In [7]:
# %pip install geocoder

In [8]:
import geocoder
import time
import io

The problem with the Geocoder package is that you have to be persistent. So we loop through the postal codes and try to get the coordinates. But we try only for times in order to avoid getting stuck.

It is possible to end up with no coordinates at all so for people with little pacience and little time skip this and pass to the next cell.

In [9]:
# toronto_df['Latitude'] = None
# toronto_df['Longitude'] = None

# for idx,row in toronto_df.iterrows():
#     # initialize your variable to None
#     tries = 4
#     lat_lng_coords = None

#     postal_code = toronto_df.PostalCode[idx]
#     print(postal_code)
#     # loop until you get the coordinates (maximum of 4 times)
#     while tries > 0:
#         g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#         lat_lng_coords = g.latlng
#         if lat_lng_coords:
#             print('Value of lat_lng_coords is:', lat_lng_coords)
#             tries = 0
#             toronto_df.loc[idx, ['Latitude', 'Longitude']] = lat_lng_coords
#         else:
#             tries -= 1
#             time.sleep(1)

To avoid having no coordinates we download the coordinates from 'http://cocl.us/Geospatial_data' and add them to our dataframe.

In [10]:
r = requests.get('http://cocl.us/Geospatial_data', stream=True)

datastr = requests.get('http://cocl.us/Geospatial_data',allow_redirects=True).text
data_file = io.StringIO(datastr)
coordinates = pd.read_csv(data_file)

coordinates.columns = ['PostalCode', 'Latitude', 'Longitude']
toronto_df = pd.merge(toronto_df[['PostalCode','Borough','Neighborhood']], coordinates, how='inner', on='PostalCode')

Let's have a look at the dataframe:

In [11]:
toronto_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


We can also check that the shape of the dataframe:

In [12]:
print('The shape of the dataframe is:', toronto_df.shape)

The shape of the dataframe is: (103, 5)


Quickly examine the resulting dataframe.

In [13]:
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [14]:
import numpy as np # library to handle data in a vectorized manner

# import pandas as pd # shoud be already imported
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# import json # library to handle JSON files

# import requests # shoud be already imported
# from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# uncomment the following line if you haven't installed it yet
# %pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# uncomment the following line if you haven't installed it yet
# %pip install folium==0.5.0
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>to_explorer</em>, as shown below.

In [15]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [16]:
# create map of New York using latitude and longitude values
map_toronto_all = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_all)  
    
map_toronto_all

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods that contain Toronto. So let's slice the original dataframe and create a new dataframe of the Toronto data.

In [17]:
toronto = toronto_df[toronto_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
print(f"The dataframe 'neighborhoods' contains {toronto.shape[0]} different postal codes")

The dataframe 'neighborhoods' contains 39 different postal codes


In [18]:
toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


We can use the same geographical coordinates as before. Now let visualize only the centre of Toronto.

In [19]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Extract Neighborhoods in Toronto

Define Foursquare Credentials and Version

In [53]:
CLIENT_ID = 'your-client-ID' # your Foursquare ID
CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your-client-ID
CLIENT_SECRET:your-client-secret


From the Foursquare lab in lab sessions, we know that all the information is in the _items_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.


In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

We have to call the API, then clean the json and structure it into a pandas dataframe. Therefore let's create a function to repeat the same process to all the neighborhoods in Toronto.

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now let's call the API and get the venues for the neighborhoods.

In [23]:
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [24]:
print(toronto_venues.shape)
toronto_venues.head()

(1577, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Let's check how many venues were returned for each neighborhood

In [25]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,59,59,59,59,59,59
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,64,64,64,64,64,64
Christie,15,15,15,15,15,15
Church and Wellesley,73,73,73,73,73,73
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,38,38,38,38,38,38
Davisville North,8,8,8,8,8,8


Let's find out how many unique categories can be curated from all the returned venues

In [26]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 231 uniques categories.


In [27]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016949,0.033898,0.000000,0.000000,0.00,0.016949,0.016949,0.000000,0.033898,0.000000,0.016949,0.016949,0.000000,0.000000,0.000000,0.00000,0.016949,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.016949,0.000000,0.000000,0.000000,0.033898,0.000000,0.000000,0.000000,0.000000,0.016949,0.050847,0.084746,0.000000,0.000000,0.00,0.00000,0.016949,0.000000,0.016949,0.000000,0.000000,0.000000,0.016949,0.000000,0.000000,0.000000,0.000000,0.016949,0.000000,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0169

Let's check the size

In [29]:
toronto_grouped.shape

(39, 231)

We are interested in finding the an ideal location for an Italian restaurant.

In [30]:
nr = len(toronto_grouped[toronto_grouped["Italian Restaurant"] > 0])
print(f'There are {nr} neighborhoods with Italian restaurants')

There are 21 neighborhoods with Italian restaurants


Let's print each neighborhood along with the top 5 most common venues

In [31]:
toronto_it = toronto_grouped[["Neighborhood","Italian Restaurant"]]
toronto_it

,Neighborhood,Italian Restaurant
0,Berczy Park,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.041667
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
3,Central Bay Street,0.046875
4,Christie,0.066667
5,Church and Wellesley,0.013699
6,"Commerce Court, Victoria Hotel",0.040000
7,Davisville,0.052632
8,Davisville North,0.000000
9,"Dufferin, Dovercourt Village",0.000000


To put everything in a pandas dataframe we will need the following function:

In [32]:
# def return_most_common_venues(row, num_top_venues):
#     row_categories = row.iloc[1:]
#     row_categories_sorted = row_categories.sort_values(ascending=False)
    
#     return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [33]:
# num_top_venues = 10

# indicators = ['st', 'nd', 'rd']

# # create columns according to number of top venues
# columns = ['Neighborhood']
# for ind in np.arange(num_top_venues):
#     try:
#         columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
#     except:
#         columns.append('{}th Most Common Venue'.format(ind+1))

# # create a new dataframe
# neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
# neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

# for ind in np.arange(toronto_grouped.shape[0]):
#     neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

# neighborhoods_venues_sorted.head()

## Cluster Neighborhoods

Run _k_-means to cluster the neighborhood into 3 clusters.

In [42]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_it.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([1, 3, 1, 3, 2, 0, 3, 2, 1, 1, 1, 0, 1, 1, 0, 0, 3, 2, 1, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [43]:
# add clustering labels

toronto_merged = toronto_it.copy()
toronto_merged.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_venues_italians = toronto_venues[toronto_venues['Venue Category']=='Italian Restaurant']

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_italians.set_index('Neighborhood'), on='Neighborhood', how='inner')
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged.head() # check the last columns!

,Cluster Labels,Neighborhood,Italian Restaurant,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
15,0,"Harbourfront East, Union Station, Toronto Islands",0.03,43.640816,-79.381752,Amano Pasta,43.644919,-79.382358,Italian Restaurant
15,0,"Harbourfront East, Union Station, Toronto Islands",0.03,43.640816,-79.381752,Piazza Manna,43.641460,-79.377719,Italian Restaurant
15,0,"Harbourfront East, Union Station, Toronto Islands",0.03,43.640816,-79.381752,Taverna Mercatto,43.642625,-79.383257,Italian Restaurant
14,0,"Garden District, Ryerson",0.02,43.657162,-79.378937,Scaddabush Italian Kitchen & Bar,43.658920,-79.382891,Italian Restaurant
14,0,"Garden District, Ryerson",0.02,43.657162,-79.378937,Trattoria Mercatto,43.654453,-79.380974,Italian Restaurant


In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Neighborhood Latitude'], toronto_merged['Neighborhood Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

# print(f'Cluster {cluster} is color {rainbow[cluster-1]}')
       
map_clusters


## Examine Clusters

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,Cluster Labels,Neighborhood,Italian Restaurant,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
15,0,"Harbourfront East, Union Station, Toronto Islands",0.030000,43.640816,-79.381752,Amano Pasta,43.644919,-79.382358,Italian Restaurant
15,0,"Harbourfront East, Union Station, Toronto Islands",0.030000,43.640816,-79.381752,Piazza Manna,43.641460,-79.377719,Italian Restaurant
15,0,"Harbourfront East, Union Station, Toronto Islands",0.030000,43.640816,-79.381752,Taverna Mercatto,43.642625,-79.383257,Italian Restaurant
14,0,"Garden District, Ryerson",0.020000,43.657162,-79.378937,Scaddabush Italian Kitchen & Bar,43.658920,-79.382891,Italian Restaurant
14,0,"Garden District, Ryerson",0.020000,43.657162,-79.378937,Trattoria Mercatto,43.654453,-79.380974,Italian Restaurant
11,0,Enclave of M5E,0.030303,43.646435,-79.374846,Metropolitan Resto Bar,43.650062,-79.377181,Italian Restaurant
11,0,Enclave of M5E,0.030303,43.646435,-79.374846,Mirto,43.650702,-79.376307,Italian Restaurant
11,0,Enclave of M5E,0.030303,43.646435,-79.374846,Carisma,43.649617,-79.375434,Italian Restaurant
29,0,St. James Town,0.025000,43.651494,-79.375418,Terroni,43.650927,-79.375602,Italian Restaurant
20,0,"Little Portugal, Trinity",0.022727,43.647927,-79.419750,Ufficio,43.649439,-79.423014,Italian Restaurant


In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,Cluster Labels,Neighborhood,Italian Restaurant,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
12,1,"First Canadian Place, Underground city",0.01,43.648429,-79.38228,Mercatto,43.650243,-79.38082,Italian Restaurant


In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,Cluster Labels,Neighborhood,Italian Restaurant,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
23,2,"Parkdale, Roncesvalles",0.066667,43.648960,-79.456325,Domani Restaurant & Wine Bar,43.649235,-79.450229,Italian Restaurant
36,2,"The Danforth West, Riverdale",0.071429,43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
36,2,"The Danforth West, Riverdale",0.071429,43.679557,-79.352188,7 Numbers,43.677062,-79.353934,Italian Restaurant
36,2,"The Danforth West, Riverdale",0.071429,43.679557,-79.352188,IL FORNELLO on Danforth,43.678604,-79.346904,Italian Restaurant
28,2,"Runnymede, Swansea",0.057143,43.651571,-79.484450,Campo,43.655191,-79.487067,Italian Restaurant
28,2,"Runnymede, Swansea",0.057143,43.651571,-79.484450,Goodfellas Wood Oven Pizza,43.648224,-79.486356,Italian Restaurant
38,2,"University of Toronto, Harbord",0.060606,43.662696,-79.400049,Daddyo's,43.664622,-79.402685,Italian Restaurant
38,2,"University of Toronto, Harbord",0.060606,43.662696,-79.400049,Piano Piano,43.662949,-79.402898,Italian Restaurant
7,2,Davisville,0.052632,43.704324,-79.388790,Positano,43.704558,-79.388639,Italian Restaurant
7,2,Davisville,0.052632,43.704324,-79.388790,Florentia Ristorante,43.703594,-79.387985,Italian Restaurant


In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3]

,Cluster Labels,Neighborhood,Italian Restaurant,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
16,3,"High Park, The Junction South",0.041667,43.661608,-79.464763,nodo,43.665303,-79.465621,Italian Restaurant
6,3,"Commerce Court, Victoria Hotel",0.040000,43.648198,-79.379817,Carisma,43.649617,-79.375434,Italian Restaurant
30,3,"St. James Town, Cabbagetown",0.040000,43.667967,-79.367675,F'Amelia,43.667536,-79.368613,Italian Restaurant
30,3,"St. James Town, Cabbagetown",0.040000,43.667967,-79.367675,Hey Lucy,43.664075,-79.368655,Italian Restaurant
6,3,"Commerce Court, Victoria Hotel",0.040000,43.648198,-79.379817,Metropolitan Resto Bar,43.650062,-79.377181,Italian Restaurant
6,3,"Commerce Court, Victoria Hotel",0.040000,43.648198,-79.379817,Terroni,43.650927,-79.375602,Italian Restaurant
6,3,"Commerce Court, Victoria Hotel",0.040000,43.648198,-79.379817,Mercatto,43.650243,-79.380820,Italian Restaurant
3,3,Central Bay Street,0.046875,43.657952,-79.387383,Tosto,43.661198,-79.386414,Italian Restaurant
3,3,Central Bay Street,0.046875,43.657952,-79.387383,Scaddabush Italian Kitchen & Bar,43.658920,-79.382891,Italian Restaurant
3,3,Central Bay Street,0.046875,43.657952,-79.387383,Mercatto,43.660391,-79.387664,Italian Restaurant


## Conclusion

In [51]:
pd.pivot_table(toronto_merged, values='Venue Category', index=['Cluster Labels'], aggfunc='count')

,Venue Category
Cluster Labels,
0,16
1,1
2,12
3,11


Cluster 1 has the fewest Restaurant with only on restaurant. This could be a possible candidate. Whereas Cluster 0 has the most. But the neighborhoods are centered around the city center, where of course the city is most densly populated and the demand is highst.

Cluster 2 & 3 are similar in size but they are located on the outskirts of the city especially cluster 2.

In conclusion this project recommends cluster 3 because one should also consider that opening restaurants in the city center may leed to high expenses like rents. Whearas in the suburbans there may no live enough potential costumers.